# Simulation notebook

In [1]:
import uuid

# Parameters cells
WITNESS_NAME = "CHSH"
SIMULATION_PATH = "./simulated_data"
MLFLOW_URL = "http://localhost:5000"
AIRFLOW_DAG_RUN_ID = str(uuid.uuid4())

In [2]:
from qutip import basis, tensor, rand_ket
import numpy as np

from entanglement_witnesses import witnesses

import mlflow

In [3]:
mlflow.set_tracking_uri(MLFLOW_URL)

In [4]:
def get_simulated_training_data(entanglement_witness, samples_nb=2000):
    samples_states = []
    samples_is_entangled = []
    
    for _ in range(0, samples_nb):
        # Create an entangled state in the form: cos(theta)|00> + exp(i*phi)*sin(theta)|11>
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2 * np.pi)
        state = (np.cos(theta) * tensor(basis(2, 0), basis(2, 0)) +
                 np.exp(1j * phi) * np.sin(theta) * tensor(basis(2, 1), basis(2, 1)))
        samples_states.append(state)
        samples_is_entangled.append(entanglement_witness(state))
    
        # Create an entangled state in the form: cos(theta)|01> + exp(i*phi)*sin(theta)|01>
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2 * np.pi)
        state = (np.cos(theta) * tensor(basis(2, 0), basis(2, 1)) +
                 np.exp(1j * phi) * np.sin(theta) * tensor(basis(2, 1), basis(2, 0)))
        samples_states.append(state)
        is_entangled = entanglement_witness(state)
        samples_is_entangled.append(is_entangled)
    
        # Create a separable state |psi> X |phi>
        state = tensor(rand_ket(2), rand_ket(2)).unit()
        samples_states.append(state)
        is_entangled = entanglement_witness(state)
        samples_is_entangled.append(is_entangled)

    return samples_states, samples_is_entangled

In [5]:
simulated_data = {}
parameter_witness = {WITNESS_NAME: witnesses[WITNESS_NAME]}

mlflow.set_experiment('ML Quantum Entanglement')

for name, witness in parameter_witness.items():
    samples_states, samples_is_entangled =  get_simulated_training_data(witness)

    simulated_data[name] = {
        "states": samples_states,
        "entanglement": samples_is_entangled
    }

In [7]:
import json

for name, data in simulated_data.items():
    states = data["states"]
    labels = np.array(data["entanglement"])
    flatten_states = np.array([np.concatenate([state.full().real.flatten(), state.full().imag.flatten()]) for state in samples_states])

    file_path = "{}/simulation-{}.npz".format(SIMULATION_PATH, name)
    np.savez(file_path, states=flatten_states, labels=labels)

    with mlflow.start_run() as run:
        mlflow.set_tag("airflow_dag_run_id", AIRFLOW_DAG_RUN_ID)
        mlflow.log_artifact(file_path, artifact_path="simulated_data")

RestException: RESOURCE_DOES_NOT_EXIST: Run '5f06f442-cf96-4c83-a89a-14e804a409ec' not found